# SF-DAT-21 | Unit Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [186]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as pl
%matplotlib inline

In [187]:
df_raw = pd.read_csv("../../dataset/admissions.csv")
df = df_raw.dropna()
df.head()

,admit,gre,gpa,prestige
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


## Part 1. Frequency Tables

#### Question 1. Let's create a frequency table of our variables.

In [188]:
# frequency table for prestige and whether or not someone was admitted
df.groupby('prestige').sum()

,admit,gre,gpa
prestige,,,
1,33,37320,210.64
2,53,88300,498.37
3,28,69560,415.38
4,12,38200,222.33


## Part 2. Return of dummy variables

#### Question 2.1. Create class or dummy variables for prestige.

In [189]:
dummies = pd.get_dummies(df.prestige, prefix = 'prestige')
dummies.head()


,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


#### Question 2.2. When modeling our class variables, how many do we need?

Answer: We need only three because the intercept will give us all the information we need for the fourth

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [190]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummies.ix[:, 'prestige_1':])
handCalc.head()

#rename because of decimal in VarName
handCalc.rename(columns = {'prestige_1.0':'prestige_1'}, inplace = True)

#hand calculate odds
print handCalc[handCalc['prestige_1'] == 1].count()
print handCalc[handCalc['prestige_1'] == 0].count()
print handCalc.prestige_1.sum()

admit           61
gre             61
gpa             61
prestige_1      61
prestige_2.0    61
prestige_3.0    61
prestige_4.0    61
dtype: int64
admit           336
gre             336
gpa             336
prestige_1      336
prestige_2.0    336
prestige_3.0    336
prestige_4.0    336
dtype: int64
61.0


In [191]:
# crosstab prestige 1 admission
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(handCalc.prestige_1, handCalc.admit, margins=True)

admit,0,1,All
prestige_1,,,
0.0,243,93,336
1.0,28,33,61
All,271,126,397


#### Question 3.1. Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college.

In [192]:
prob_admit_rank1 = 33/61.0
prob_reject_rank1 = 1 - prob_admit_rank1
odds_admit_rank1 = prob_admit_rank1/prob_reject_rank1
print odds_admit_rank1

1.17857142857


#### Question 3.2. Now calculate the odds of admission if you did not attend a #1 ranked college.

In [193]:
prob_admit_notrank1 = 93/336.0
prob_reject_notrank1 = 1 - prob_admit_notrank1
odds_admit_notrank1 = prob_admit_notrank1/prob_reject_notrank1
print odds_admit_notrank1

0.382716049383


#### Question 3.3. Calculate the odds ratio.

In [194]:
odds_ratio = odds_admit_rank1/odds_admit_notrank1
print odds_ratio

3.07949308756


#### Question 3.4. Write this finding in a sentenance:

Answer: The odds of being admitted from rank one school are three times that of being admitted from a lower rank school

#### Question 3.5. Print the cross tab for prestige_4.

In [195]:
#rename because of decimal in VarName
handCalc.rename(columns = {'prestige_4.0':'prestige_4'}, inplace = True)
pd.crosstab(handCalc.prestige_4, handCalc.admit, margins=True)

admit,0,1,All
prestige_4,,,
0.0,216,114,330
1.0,55,12,67
All,271,126,397


#### Question 3.6. Calculate the OR.

In [196]:
prob_admit_rank4 = 12/67.0
prob_reject_rank4 = 1 - prob_admit_rank4
odds_admit_rank4 = prob_admit_rank4/prob_reject_rank4
print odds_admit_rank4

prob_admit_notrank4 = 114/330.0
prob_reject_notrank4 = 1 - prob_admit_notrank4
odds_admit_notrank4 = prob_admit_notrank4/prob_reject_notrank4
print odds_admit_notrank4

odds_ratio_4 = odds_admit_rank4/odds_admit_notrank4
print odds_ratio_4

#get the inverse
print 1/odds_ratio_4

0.218181818182
0.527777777778
0.413397129187
2.41898148148


#### Question 3.7. Write this finding in a sentence.

Answer: The odds of being admitted and not coming from a rank 4 school is 2.4 times the odds of being admitted from a rank 4 school

## Part 4. Analysis

In [197]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummies.ix[:, 'prestige_2':])
print data.head()

   admit  gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380  3.61             0             1             0
1      1  660  3.67             0             1             0
2      1  800  4.00             0             0             0
3      1  640  3.19             0             0             1
4      0  520  2.93             0             0             1


We're going to add a constant term for our Logistic Regression.  The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [198]:
# manually add the intercept
data['intercept'] = 1.0
data.head()

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0,intercept
0,0,380,3.61,0,1,0,1
1,1,660,3.67,0,1,0,1
2,1,800,4.00,0,0,0,1
3,1,640,3.19,0,0,1,1
4,0,520,2.93,0,0,1,1


#### Question 4.1. Set the covariates to a variable called train_cols.

In [199]:
train_cols = ['intercept', 'gre', 'gpa', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']
dep = data['admit']
ind = data[train_cols]

#### Question 4.2. Fit the model.

In [201]:
logit_model = sm.Logit(dep, ind)
logit_fit = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### Question 4.3. Print the summary results.

In [202]:
print logit_fit.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Tue, 29 Mar 2016   Pseudo R-squ.:                 0.08166
Time:                        17:55:47   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
intercept       -3.8769      1.142     -3.393      0.001        -6.116    -1.638
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.34

#### Question 4.4. Calculate the odds ratios of the coeffincients and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params

        conf.columns = ['2.5%', '97.5%', 'OR']

In [208]:
print(np.exp(logit_fit.params))


intercept       0.020716
gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
dtype: float64


In [209]:
conf = logit_fit.conf_int()
params = logit_fit.params

conf['OR'] = params

#rename conf interval columns
conf.columns = ['2.5%', '97.5%', 'OR']

#normalize
norm_conf = np.exp(conf)
print norm_conf

                  2.5%     97.5%        OR
intercept     0.002207  0.194440  0.020716
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525


#### Question 4.5. Interpret the OR of Prestige_2.

Answer: This means that the odds of getting admitted with a rank 1 school is .5 the odds of getting addmitted with a rank 1 school

#### Question 4.6. Interpret the OR of GPA.

Answer: the odds of getting admitted with a higher GPA is 2.18 the odds of getting in with a lower GPA (lower by one increment?)

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values.  This will allow us to see how the predicted probability of admission increases/decreases across different variables.  First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa".  This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [230]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [231]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)

print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])

gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)

print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])

# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))
combos.head()

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


,0,1,2,3
0,220,2.260000,1,1
1,220,2.260000,2,1
2,220,2.260000,3,1
3,220,2.260000,4,1
4,220,2.453333,1,1


#### Question 5.1. Recreate the dummy variables.

In [232]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
combos.head()

dummies_2 = pd.get_dummies(combos.prestige, prefix = 'prestige')
dummies_2.head()
#rename dummies all at once
dummies_2.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']
dummies_2.head()

# keep only what we need for making predictions
combos_join = combos.join(dummies_2.ix[:, 'prestige_1':])
combos_join.head()

,gre,gpa,prestige,intercept,prestige_1,prestige_2,prestige_3,prestige_4
0,220,2.260000,1,1,1,0,0,0
1,220,2.260000,2,1,0,1,0,0
2,220,2.260000,3,1,0,0,1,0
3,220,2.260000,4,1,0,0,0,1
4,220,2.453333,1,1,1,0,0,0


#### Question 5.2. Make predictions on the enumerated dataset.

In [233]:

ind = combos_join[['gre', 'gpa', 'prestige_2','prestige_3','prestige_4','intercept']]

combos_join['pred'] = logit_fit.predict(ind)
combos_join.head()
combos_join.tail()
#unable to get predictions. What is the error?

,gre,gpa,prestige,intercept,prestige_1,prestige_2,prestige_3,prestige_4,pred
395,800,3.806667,4,1,0,0,0,1,0
396,800,4.000000,1,1,1,0,0,0,0
397,800,4.000000,2,1,0,1,0,0,0
398,800,4.000000,3,1,0,0,1,0,0
399,800,4.000000,4,1,0,0,0,1,0


#### Question 5.3. Interpret findings for the last 4 observations.

Answer: -

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.